Qwen

In [1]:
from unsloth import FastModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.lm_head.load_state_dict(torch.load("../../Model/qwen16.pth"))
model.model.embed_tokens.load_state_dict(torch.load("../../Model/qwen16.pth"))
model.train();
model.lm_head.weight.requires_grad_(True);
model.model.embed_tokens.weight.requires_grad_(True);

In [3]:
# resize model vocab
# model.model.embed_tokens = Gemma3TextScaledWordEmbedding(16,2560,0,50.59644256269407)
# model.lm_head = nn.Linear(2560, 16, bias=False)
# model.lm_head.weight = model.model.embed_tokens.weight

In [3]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [4]:
model.config._attn_implementation

'sdpa'

In [21]:
random_attention_mask[0,0]

tensor([[-0.7424,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
        [ 0.4867, -0.4144,    -inf,  ...,    -inf,    -inf,    -inf],
        [-0.0248,  1.4264,  0.6635,  ...,    -inf,    -inf,    -inf],
        ...,
        [-0.5023,  0.1269, -1.4265,  ..., -1.7798,    -inf,    -inf],
        [-0.3270,  0.7548, -0.1801,  ..., -0.0637, -0.5220,    -inf],
        [-0.8897,  1.4543,  0.7084,  ..., -1.0433,  0.4343,  1.7050]],
       device='cuda:0')

In [19]:
# check for causality
inputs = next(iter(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)))
seq_length = inputs['input_tokens'].shape[1]
random_values = torch.randn(1, 32, seq_length, seq_length, device='cuda')
# Create a causal attention mask where future tokens are masked out with -inf
# Shape: [1, 32, seq_length, seq_length]
causal_mask = torch.zeros(1, 32, seq_length, seq_length, device='cuda', dtype=torch.float32)
# Fill the upper triangular part (future tokens) with -inf
causal_mask = causal_mask.masked_fill(
    torch.triu(torch.ones(seq_length, seq_length, device='cuda'), diagonal=1).bool().unsqueeze(0).unsqueeze(0),
    float('-inf')
)
# Convert to bfloat16 for compatibility with model operations
random_attention_mask = causal_mask + random_values

with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        out = model(inputs['input_tokens'], 
                    attention_mask=random_attention_mask,
                    )
logits1 = out.logits
print(inputs['input_tokens'][0][256])
inputs['input_tokens'][0][256] += 1
print(inputs['input_tokens'][0][256])
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        out = model(inputs['input_tokens'], 
                    attention_mask=random_attention_mask,
                    )
logits2 = out.logits
print(torch.where(torch.max(torch.abs(logits1 - logits2)[0],dim=1).values == 0))

tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
(tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        1

In [22]:
inputs = next(iter(data_gen(data, IsTrain=False, max_length=8192, autoregressive=True, NeedPosition=False, tokenize_func=tokenize_causal, IsDecode=False)))
seq_length = inputs['input_tokens'].shape[1]
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        out = model(inputs['input_tokens'], 
                    attention_mask=random_attention_mask,
                    )
logits1 = out.logits
# random_attention_mask = torch.randn(1, 32, seq_length, seq_length, device='cuda', dtype=torch.bfloat16) * 1000
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        out = model(inputs['input_tokens'], 
                    # attention_mask=random_attention_mask,
                    )
logits2 = out.logits
print(torch.mean(torch.abs(logits1 - logits2)[0]))

tensor(0.7656, device='cuda:0', dtype=torch.bfloat16)
